In [1]:
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
train_df = pd.read_csv("../../input/feedback-prize-english-language-learning/train.csv")
train_df.head(2)

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5


In [3]:
model_path_list = [
    #('../../07_Embeddings/exp/result/07_v1_01/oof_df.csv',1), # svr
    
    #('../../14_Baseline4/exp/result/14_v1_01/oof_df.csv',1), # deberta-v3-base
    ('../../14_Baseline4/exp/result/14_v1_07/oof_df.csv',1), # deberta-v3-base, 10folds     
    #('../../14_Baseline4/exp/result/14_v1_08/oof_df.csv',1), # deberta-base
    #('../../14_Baseline4/exp/result/14_v1_09/oof_df.csv',1), # roberta-base
    ('../../14_Baseline4/exp/result/14_v1_10/oof_df.csv',1), # deberta-v3-large
    #('../../14_Baseline4/exp/result/14_v1_11/oof_df.csv',1), # xlm-roberta-base
    ('../../14_Baseline4/exp/result/14_v1_12/oof_df.csv',1), # deberta-large
    #('../../14_Baseline4/exp/result/14_v1_13/oof_df.csv',1), # roberta-large
    #('../../14_Baseline4/exp/result/14_v1_14/oof_df.csv',1), # xlm-roberta-large
#     ('../../14_Baseline4/exp/result/14_v1_15/oof_df.csv',1), # deberta-v2-xlarge
#     ('../../14_Baseline4/exp/result/14_v1_16/oof_df.csv',1), # deberta-xlarge
    
#     ('../../14_Baseline4/exp/result/14_v2_01/oof_df.csv',1), # deberta-v3-base, seed200
#     ('../../14_Baseline4/exp/result/14_v2_02/oof_df.csv',1), # deberta-v3-large, seed200
#     ('../../14_Baseline4/exp/result/14_v2_03/oof_df.csv',1), # deberta-base, seed200
    
    #('../../17_Pseudo3/exp/result/17_v1_01/oof_df.csv',1), # deberta-v3-base
    #('../../17_Pseudo3/exp/result/17_v1_02/oof_df.csv',1), # deberta-v3-base
    #('../../17_Pseudo3/exp/result/17_v1_03/oof_df.csv',1), # deberta-v3-large
    #('../../17_Pseudo3/exp/result/17_v1_04/oof_df.csv',1), # deberta-v3-base, seed200
    #('../../17_Pseudo3/exp/result/17_v1_05/oof_df.csv',1), # deberta-base
    #('../../17_Pseudo3/exp/result/17_v1_06/oof_df.csv',1), # deberta-large
    #('../../17_Pseudo3/exp/result/17_v1_07/oof_df.csv',1), # deberta-v3-small
    
    #('../../18_Pseudo4/exp/result/18_v1_01/oof_df.csv',1), # deberta-v3-base
    #('../../18_Pseudo4/exp/result/18_v1_02/oof_df.csv',1), # deberta-v3-large
    #('../../18_Pseudo4/exp/result/18_v1_03/oof_df.csv',1), # deberta-v3-small
    
    #('../../19_Distillation/exp/result/19_v1_01/oof_df.csv',1), # deberta-v3-base <- dbv3-l
    ('../../19_Distillation/exp/result/19_v1_02/oof_df.csv',1), # deberta-v3-base <- dbv3-xl
    ('../../19_Distillation/exp/result/19_v1_03/oof_df.csv',1), # deberta-v3-base <- db-xl
    #('../../19_Distillation/exp/result/19_v1_04/oof_df.csv',1), # deberta-v3-small <- dbv3-l
    #('../../19_Distillation/exp/result/19_v1_05/oof_df.csv',1), # deberta-v3-base <- db-l
    
    # single models
    ('../../20_Distillation2/exp/result/20_v2_01/oof_df.csv',1), # deberta-v3-base <- dbv3-b
    #('../../20_Distillation2/exp/result/20_v2_02/oof_df.csv',1), # deberta-v3-base <- dbv3-l
    #('../../20_Distillation2/exp/result/20_v2_03/oof_df.csv',1), # deberta-v3-large <- dbv3-l
    #('../../20_Distillation2/exp/result/20_v2_04/oof_df.csv',1), # deberta-base <- db-b
    #('../../20_Distillation2/exp/result/20_v2_05/oof_df.csv',1), # deberta-v3-base <- db-b
    #('../../20_Distillation2/exp/result/20_v2_06/oof_df.csv',1), # deberta-v3-base <- db-l
    #('../../20_Distillation2/exp/result/20_v2_07/oof_df.csv',1), # deberta-v3-base <- svr
    #('../../20_Distillation2/exp/result/20_v2_08/oof_df.csv',1), # deberta-v3-base <- xlm-rb-b
    ('../../20_Distillation2/exp/result/20_v2_09/oof_df.csv',1), # deberta-v3-base <- xlm-rb-l
]

oof_df_list = [
    pd.read_csv(model_path) for model_path,_ in model_path_list
]
weights_list = [
    w for _,w in model_path_list
]
weights_list = [w/sum(weights_list) for w in weights_list]

for oof_df in oof_df_list:
    oof_df = train_df[['text_id']].merge(oof_df, how='left', on='text_id')
    
num_models = len(model_path_list)

TARGET_COLS = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']

preds = 0
for oof_df, w in zip(oof_df_list, weights_list):
    preds += oof_df[TARGET_COLS].values * w

oofs = []
for oof_df in oof_df_list:
    oofs.append(oof_df[TARGET_COLS].values)

In [4]:
oof_df = oof_df_list[0].copy()
for i,col in enumerate(TARGET_COLS):
    oof_df[col] = preds[:,i]

In [5]:
import numpy as np

def calc_metric(pred, gt):
    '''
    pred : (num_data, num_labels)
    gt : (num_data, num_labels)
    '''
    score = np.sqrt(np.mean((pred - gt)**2, axis=0))
    score = score.mean()
    return score

In [6]:
score = calc_metric(pred=oof_df[TARGET_COLS].values, gt=train_df[TARGET_COLS].values)
print('CV={:.4f}'.format(score))

CV=0.4437


# Optimize class-wise

In [7]:
ensemble_predictions=np.stack(oofs)
ensemble_predictions.shape

(7, 3911, 6)

In [8]:
from skopt import gp_minimize

def run_optimize(i):
    target_col = TARGET_COLS[i]
    print("*"*50)
    print(target_col)
    print("*"*50)
    def ensemble_score(weights,return_pred=False):
        weights=np.array(weights)
        weights=weights.reshape(-1,1,1)/weights.sum()
        p=weights.reshape(-1,1,1)*ensemble_predictions[:,:,i,None]
        p=p.sum(0)
        score=calc_metric(p, train_df[[target_col]])
        if return_pred:
            return score,p
        else:
            return score   
    results = gp_minimize(ensemble_score, np.array([[0.1,1] for i in range(len(ensemble_predictions))]),
                          verbose=True,random_state=2022)
    best_weights=np.array(results['x'])/sum(results['x'])
    print(best_weights)
    # smaller is better for this metric
    score, ensemble_pred=ensemble_score(best_weights,True)
    print("score={:.4f}", score)
    return score, best_weights

In [9]:
scores = {}
best_weights = {}
for i,target_col in enumerate(TARGET_COLS):
    score, weights = run_optimize(i)
    scores[target_col] = score
    best_weights[target_col] = weights

**************************************************
cohesion
**************************************************
Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.0048
Function value obtained: 0.4720
Current minimum: 0.4720
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0022
Function value obtained: 0.4720
Current minimum: 0.4720
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.0018
Function value obtained: 0.4743
Current minimum: 0.4720
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.0018
Function value obtained: 0.4719
Current minimum: 0.4719
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.

Iteration No: 41 ended. Search finished for the next optimal point.
Time taken: 0.3164
Function value obtained: 0.4711
Current minimum: 0.4711
Iteration No: 42 started. Searching for the next optimal point.
Iteration No: 42 ended. Search finished for the next optimal point.
Time taken: 0.3523
Function value obtained: 0.4734
Current minimum: 0.4711
Iteration No: 43 started. Searching for the next optimal point.
Iteration No: 43 ended. Search finished for the next optimal point.
Time taken: 0.3358
Function value obtained: 0.4730
Current minimum: 0.4711
Iteration No: 44 started. Searching for the next optimal point.
Iteration No: 44 ended. Search finished for the next optimal point.
Time taken: 0.3662
Function value obtained: 0.4742
Current minimum: 0.4711
Iteration No: 45 started. Searching for the next optimal point.
Iteration No: 45 ended. Search finished for the next optimal point.
Time taken: 0.3934
Function value obtained: 0.4736
Current minimum: 0.4711
Iteration No: 46 started. Sea

Iteration No: 81 ended. Search finished for the next optimal point.
Time taken: 0.5817
Function value obtained: 0.4723
Current minimum: 0.4710
Iteration No: 82 started. Searching for the next optimal point.
Iteration No: 82 ended. Search finished for the next optimal point.
Time taken: 0.6198
Function value obtained: 0.4710
Current minimum: 0.4710
Iteration No: 83 started. Searching for the next optimal point.
Iteration No: 83 ended. Search finished for the next optimal point.
Time taken: 0.5992
Function value obtained: 0.4719
Current minimum: 0.4710
Iteration No: 84 started. Searching for the next optimal point.
Iteration No: 84 ended. Search finished for the next optimal point.
Time taken: 0.5859
Function value obtained: 0.4719
Current minimum: 0.4710
Iteration No: 85 started. Searching for the next optimal point.
Iteration No: 85 ended. Search finished for the next optimal point.
Time taken: 0.6076
Function value obtained: 0.4710
Current minimum: 0.4710
Iteration No: 86 started. Sea

Iteration No: 21 ended. Search finished for the next optimal point.
Time taken: 0.2227
Function value obtained: 0.4376
Current minimum: 0.4366
Iteration No: 22 started. Searching for the next optimal point.
Iteration No: 22 ended. Search finished for the next optimal point.
Time taken: 0.2346
Function value obtained: 0.4367
Current minimum: 0.4366
Iteration No: 23 started. Searching for the next optimal point.
Iteration No: 23 ended. Search finished for the next optimal point.
Time taken: 0.2146
Function value obtained: 0.4374
Current minimum: 0.4366
Iteration No: 24 started. Searching for the next optimal point.
Iteration No: 24 ended. Search finished for the next optimal point.
Time taken: 0.2524
Function value obtained: 0.4374
Current minimum: 0.4366
Iteration No: 25 started. Searching for the next optimal point.
Iteration No: 25 ended. Search finished for the next optimal point.
Time taken: 0.2780
Function value obtained: 0.4372
Current minimum: 0.4366
Iteration No: 26 started. Sea

Iteration No: 61 ended. Search finished for the next optimal point.
Time taken: 0.4304
Function value obtained: 0.4375
Current minimum: 0.4365
Iteration No: 62 started. Searching for the next optimal point.
Iteration No: 62 ended. Search finished for the next optimal point.
Time taken: 0.4587
Function value obtained: 0.4379
Current minimum: 0.4365
Iteration No: 63 started. Searching for the next optimal point.
Iteration No: 63 ended. Search finished for the next optimal point.
Time taken: 0.4384
Function value obtained: 0.4376
Current minimum: 0.4365
Iteration No: 64 started. Searching for the next optimal point.
Iteration No: 64 ended. Search finished for the next optimal point.
Time taken: 0.4916
Function value obtained: 0.4384
Current minimum: 0.4365
Iteration No: 65 started. Searching for the next optimal point.
Iteration No: 65 ended. Search finished for the next optimal point.
Time taken: 0.4804
Function value obtained: 0.4379
Current minimum: 0.4365
Iteration No: 66 started. Sea

...

Iteration No: 10 ended. Evaluation done at random point.
Time taken: 0.2259
Function value obtained: 0.4650
Current minimum: 0.4629
Iteration No: 11 started. Searching for the next optimal point.
Iteration No: 11 ended. Search finished for the next optimal point.
Time taken: 0.1899
Function value obtained: 0.4634
Current minimum: 0.4629
Iteration No: 12 started. Searching for the next optimal point.
Iteration No: 12 ended. Search finished for the next optimal point.
Time taken: 0.3028
Function value obtained: 0.4641
Current minimum: 0.4629
Iteration No: 13 started. Searching for the next optimal point.
Iteration No: 13 ended. Search finished for the next optimal point.
Time taken: 0.2330
Function value obtained: 0.4622
Current minimum: 0.4622
Iteration No: 14 started. Searching for the next optimal point.
Iteration No: 14 ended. Search finished for the next optimal point.
Time taken: 0.2313
Function value obtained: 0.4632
Current minimum: 0.4622
Iteration No: 15 started. Searching for 

Iteration No: 50 ended. Search finished for the next optimal point.
Time taken: 0.3839
Function value obtained: 0.4640
Current minimum: 0.4622
Iteration No: 51 started. Searching for the next optimal point.
Iteration No: 51 ended. Search finished for the next optimal point.
Time taken: 0.3849
Function value obtained: 0.4640
Current minimum: 0.4622
Iteration No: 52 started. Searching for the next optimal point.
Iteration No: 52 ended. Search finished for the next optimal point.
Time taken: 0.4237
Function value obtained: 0.4640
Current minimum: 0.4622
Iteration No: 53 started. Searching for the next optimal point.
Iteration No: 53 ended. Search finished for the next optimal point.
Time taken: 0.4075
Function value obtained: 0.4628
Current minimum: 0.4622
Iteration No: 54 started. Searching for the next optimal point.
Iteration No: 54 ended. Search finished for the next optimal point.
Time taken: 0.3900
Function value obtained: 0.4622
Current minimum: 0.4622
Iteration No: 55 started. Sea

Iteration No: 90 ended. Search finished for the next optimal point.
Time taken: 0.7097
Function value obtained: 0.4633
Current minimum: 0.4621
Iteration No: 91 started. Searching for the next optimal point.
Iteration No: 91 ended. Search finished for the next optimal point.
Time taken: 0.7728
Function value obtained: 0.4630
Current minimum: 0.4621
Iteration No: 92 started. Searching for the next optimal point.
Iteration No: 92 ended. Search finished for the next optimal point.
Time taken: 0.7839
Function value obtained: 0.4621
Current minimum: 0.4621
Iteration No: 93 started. Searching for the next optimal point.
Iteration No: 93 ended. Search finished for the next optimal point.
Time taken: 0.8378
Function value obtained: 0.4640
Current minimum: 0.4621
Iteration No: 94 started. Searching for the next optimal point.
Iteration No: 94 ended. Search finished for the next optimal point.
Time taken: 0.8161
Function value obtained: 0.4621
Current minimum: 0.4621
Iteration No: 95 started. Sea

Iteration No: 30 ended. Search finished for the next optimal point.
Time taken: 0.2670
Function value obtained: 0.4371
Current minimum: 0.4370
Iteration No: 31 started. Searching for the next optimal point.
Iteration No: 31 ended. Search finished for the next optimal point.
Time taken: 0.2479
Function value obtained: 0.4372
Current minimum: 0.4370
Iteration No: 32 started. Searching for the next optimal point.
Iteration No: 32 ended. Search finished for the next optimal point.
Time taken: 0.3047
Function value obtained: 0.4370
Current minimum: 0.4370
Iteration No: 33 started. Searching for the next optimal point.
Iteration No: 33 ended. Search finished for the next optimal point.
Time taken: 0.3557
Function value obtained: 0.4372
Current minimum: 0.4370
Iteration No: 34 started. Searching for the next optimal point.
Iteration No: 34 ended. Search finished for the next optimal point.
Time taken: 0.2993
Function value obtained: 0.4371
Current minimum: 0.4370
Iteration No: 35 started. Sea

Iteration No: 70 ended. Search finished for the next optimal point.
Time taken: 0.5040
Function value obtained: 0.4373
Current minimum: 0.4369
Iteration No: 71 started. Searching for the next optimal point.
Iteration No: 71 ended. Search finished for the next optimal point.
Time taken: 0.5312
Function value obtained: 0.4384
Current minimum: 0.4369
Iteration No: 72 started. Searching for the next optimal point.
Iteration No: 72 ended. Search finished for the next optimal point.
Time taken: 0.5073
Function value obtained: 0.4369
Current minimum: 0.4369
Iteration No: 73 started. Searching for the next optimal point.
Iteration No: 73 ended. Search finished for the next optimal point.
Time taken: 0.5296
Function value obtained: 0.4369
Current minimum: 0.4369
Iteration No: 74 started. Searching for the next optimal point.
Iteration No: 74 ended. Search finished for the next optimal point.
Time taken: 0.5172
Function value obtained: 0.4373
Current minimum: 0.4369
Iteration No: 75 started. Sea

In [10]:
best_weights

{'cohesion': array([0.30106896, 0.15550464, 0.27473294, 0.13474377, 0.0737359 ,
        0.0301069 , 0.0301069 ]),
 'syntax': array([0.23622479, 0.20422289, 0.26091187, 0.13709647, 0.02609119,
        0.02609119, 0.1093616 ]),
 'vocabulary': array([0.25628874, 0.31454857, 0.16390538, 0.05152633, 0.03145486,
        0.12595287, 0.05632324]),
 'phraseology': array([0.21668767, 0.19347292, 0.1494323 , 0.14789026, 0.07511655,
        0.0608319 , 0.1565684 ]),
 'grammar': array([0.23239898, 0.36999143, 0.03699914, 0.10740043, 0.11873459,
        0.03699914, 0.09747629]),
 'conventions': array([0.24238728, 0.16816649, 0.17451642, 0.17465133, 0.19180103,
        0.02423873, 0.02423873])}

In [11]:
scores

{'cohesion': 0.471010959905199,
 'syntax': 0.43650835600791066,
 'vocabulary': 0.4051137238465872,
 'phraseology': 0.445761011351948,
 'grammar': 0.46211384331482475,
 'conventions': 0.43692759089703526}

In [12]:
CV = sum(scores.values()) / len(scores)
print("CV={:.4f}".format(CV))

CV=0.4429


# Error Analysis - Check Corr

In [13]:
cols = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']
train_df[cols].corr()

,cohesion,syntax,vocabulary,phraseology,grammar,conventions
cohesion,1.000000,0.695459,0.666151,0.690058,0.638689,0.666151
syntax,0.695459,1.000000,0.680562,0.725467,0.709525,0.700025
vocabulary,0.666151,0.680562,1.000000,0.735261,0.654852,0.664292
phraseology,0.690058,0.725467,0.735261,1.000000,0.719746,0.666842
grammar,0.638689,0.709525,0.654852,0.719746,1.000000,0.673301
conventions,0.666151,0.700025,0.664292,0.666842,0.673301,1.000000


In [14]:
cols = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']
oof_df[cols].corr()

,cohesion,syntax,vocabulary,phraseology,grammar,conventions
cohesion,1.000000,0.954818,0.928050,0.922189,0.836890,0.896597
syntax,0.954818,1.000000,0.949594,0.969260,0.929517,0.933198
vocabulary,0.928050,0.949594,1.000000,0.965852,0.886963,0.902633
phraseology,0.922189,0.969260,0.965852,1.000000,0.949663,0.893236
grammar,0.836890,0.929517,0.886963,0.949663,1.000000,0.865892
conventions,0.896597,0.933198,0.902633,0.893236,0.865892,1.000000
